This notebook performs some rudimentary Network Signal Strength Analytics. 
It uses data from the app on iOS devcies call RF Checker. This creates a .csv file of the signal strength of the RF antenae

First we install some libraries
The beginnings of this notebook came from https://georgetsilva.github.io/posts/mapping-points-with-folium/

# Step 1

Load Libraries

In [12]:
import pandas as pd
import requests
from xml.etree import ElementTree
import numpy as np

DSX does not by default have Folium installed. This installs it

In [13]:
import sys
try:
    import folium
except:
    if hasattr(sys, 'real_prefix'):
        #we are in a virtual env.
        !pip install folium 
    else:
        !pip install --user folium
    import folium

## Step 2
load data from CSV File - eventually this will come from a cloudant, or dash DB using Data Connect

### Sequence for Data Import

File transformation
+ RF_Checker_2017<mnth><day>_<time> - This is the original file from RF Checker data and time stamped. Eventually this will be many files that nned to be imported into DashDB (DB2 Warehouse on the Cloud)
+ Using Data Connect we trasnfer each of the RF_Checker data into DashDB. This removes a lot of unnecessary columns from the .csv files and changes a few column titles to be more meanigful
+ Once in DashDB (and we have added the connection to this project) we import that into a 


In [14]:
# The code was removed by DSX for sharing.

,Time,Signal Strength,Gps_Latitude,Gps_Longitude,Gps_Height,Gps_Horizontal_Accuracy,Gps_Vertical_Accuracy,Gps_Speed
0,2017-08-31 12:14:17,-85.0,39.698617,-104.957078,1633.825029,10.0,3.0,NaN
1,2017-08-31 12:14:18,-85.0,39.698624,-104.957060,1633.988115,10.0,3.0,1.09
2,2017-08-31 12:14:19,-85.0,39.698628,-104.957044,1633.839434,10.0,3.0,1.16
3,2017-08-31 12:14:20,-85.0,39.698634,-104.957032,1633.756670,10.0,3.0,1.09
4,2017-08-31 12:14:21,-85.0,39.698640,-104.957022,1633.702959,10.0,3.0,0.88


## Step 3
Now that we have the data in a DataFrame 

However, The RF Checker data records a row approx every second, so we need to group signal strength in blocks of X seconds 

In [15]:
# extract the specific data from DashDB
signalCoordinates = data_df_1[['Time', 'Signal Strength', 'Gps_Latitude', 'Gps_Longitude']]
timeIndex = 0
strengthIndex = 1
latIndex = 2
longIndex = 3
# create a list of data
unfilteredLocationList = signalCoordinates.values.tolist()

# Used to find the center of the map
lengthOfList = float(len(unfilteredLocationList))
# find average lat
coords = [ item[2] for item in unfilteredLocationList ]
lat = sum(coords) / lengthOfList
print(lat)
# find average Long
coords = [ item[3] for item in unfilteredLocationList ]
long = sum(coords) / lengthOfList
print(long)
centerOfMap = [lat, long]

39.69942947989455
-104.95556064601156


Create a list of GPS Elements, that contain a list of GPS coordinates

In [16]:
# GPSElement = {
#   'Strength' : aDBmValue, 
#   'Coordinates' : [['lat' : aValue , 'long': aValue]], 
#   'SignalCount' : aNumber }

GPSElementList = []  # this will contain the list of GPSElements to display on the map
import math
# creates a GPSElement from an unfiltered DashDB row
def createGPSElement(anUnfilteredRow):
    coord = []
    coord.append([ anUnfilteredRow[latIndex], anUnfilteredRow[longIndex]]) 
    return {'Strength' : anUnfilteredRow[strengthIndex],
         'SignalCount' : 0,
         'Coordinates' : coord
    }

# Appends the coordinates of the unfileteredRow to the GPSElement
def addGPSCoordinate(aGPSElement, anUnfilteredRow):
    aGPSElement['Coordinates'].append([anUnfilteredRow[latIndex], anUnfilteredRow[longIndex]])
    aGPSElement['SignalCount'] = aGPSElement['SignalCount'] + 1
    return aGPSElement

# Assuming the unfilteredlist is sorted into groups of singal strength, 
# when the signal strength changes we start another GPSElement
for x in range(0, len(unfilteredLocationList)):
    unfilteredRow = unfilteredLocationList[x]
    aSignalStrength = unfilteredRow[strengthIndex]
    if math.isnan(aSignalStrength) :
        continue
    if len(GPSElementList) == 0 or aSignalStrength != latestGPSElement['Strength']:
        latestGPSElement = createGPSElement(unfilteredRow)
        GPSElementList.append(latestGPSElement)
    else:
        addGPSCoordinate(latestGPSElement,unfilteredRow)
 

Plot the coodinates on the map
Center the map on the coordinate of the first row's lat, long 

I am assuming at this point we have an array of rows to plot. Each row contains 
+ Signal Strength
+ Array of Lat, Long positions thst will be plotted - this might be less than the nukmber of points that were counted for this signal Strength
+ Number with signal Strength

Using this array I am going to select a color for the signal Strength and then plot a polygon using the array of Lat Log, and a label with the # of points tht meet the match

Need a color scale for the RF Signal Strength. Use Branca

In [23]:
import branca.colormap as cm
# find lowest signal Strength
low = min([ item['Strength'] for item in GPSElementList])
# find highest signal Strength
high = max([ item['Strength'] for item in GPSElementList])
colorScale = cm.linear.PuOr.scale(low, high)
colorScale

Now I can plot on Map

In [24]:
# create a map, center it on one of the coordinates, and set zoom level
map = folium.Map(location=centerOfMap, zoom_start=18)

for row in range(0, len(GPSElementList)):
    signalStrength = GPSElementList[row]['Strength']
    coordinates = GPSElementList[row]['Coordinates']
    signalStrengthCount = GPSElementList[row]['SignalCount']
    # display a polygon withthe coordinates
    folium.PolyLine(coordinates,
                    color=colorScale(signalStrength), 
                    weigth=10, 
                    opacity=0.8).add_to(map)  
map